!pip install numpy==1.21.6 pandas==1.3.5 matplotlib==3.5.3 seaborn==0.12.2 chainladder==0.8.18  numba==0.56.4

# Algoritmo para calcular taxa de retenção
Esse notebook contempla a análise exploratória sobre a retenção de profissionais nas regiões de saúde com base em dados do CNES-PF. O notebook contém o script principal e métodos auxiliares, que ajudam a tratar os dados e criar algumas visualizações.

Observação:
```
Antes de executar o notebook é necessário criar 2 pastas:
- imgs
- csvs
```

## Carregar bibliotecas

In [1]:
import traceback
import warnings
from operator import attrgetter
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chainladder as cl

## Configurar bibliotecas

In [2]:
# Remover limite de exibição de linhas e colunas do pandas
pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

# Evitar notações científicas no pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Evitar warnings sobre perfomance no pandas
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Funções auxiliares para tratamento dos dados

In [3]:
def calcular_saida(row):
    old_values = row.copy()
    for col in row.index:
        try:
            if col == 180:
                row[col] = None
            else:
                if col < 12:
                    row[col] = row[col] - old_values[col + 12]
                else:
                    row[col] = (row[col] - old_values[col + 12]) + row[col - 12]
        except:
            pass
    return row


def gerar_triangulo(df):
    saidas = pd.melt(df.copy().apply(calcular_saida, axis=1).reset_index().drop([180], axis=1),
                     id_vars=['cohort'], value_name='values', var_name='period_number').sort_values(
        ["cohort", 'period_number'])
    saidas['origin'] = saidas['cohort'].dt.year
    saidas['development'] = (saidas['cohort'] + saidas['period_number'])
    saidas['development'] = saidas['development'].apply(lambda x: datetime.strptime(str(x), '%Y-%m').year)
    saidas['dif'] = saidas['development'] - saidas['origin']
    saidas.sort_values(['dif', 'origin'], inplace=True)
    saidas = saidas[['development', 'origin', 'values']]
    saidas.dropna(inplace=True)

    return saidas


def gerar_triangulo_agregado(df_tmp):
    # Converter a coluna competência para o formato ANO-MES (datetime)
    df_tmp['COMPETEN'] = pd.to_datetime(df_tmp['COMPETEN'], format='%Y%m').dt.to_period('M')

    # Criar o grupo no qual o profissional pertence (sua primeira competência)
    df_tmp['cohort'] = df_tmp.groupby('CPF_PROF')['COMPETEN'].transform('min')

    # Contar a quantidade de profissionais distintos por CPF para cada grupo e competência
    df_cohort = df_tmp.groupby(['uf_sigla', 'cod_regsaud', 'cohort', 'COMPETEN']).agg(
        n_prof=('CPF_PROF', 'nunique')).reset_index(drop=False)

    # Eliminar o primeiro grupo
    df_cohort = df_cohort[df_cohort['cohort'] != '2008-01']

    # Calcular a diferença em meses entre a competência e o grupo (primeira competência)
    df_cohort['period_number'] = (df_cohort.COMPETEN - df_cohort.cohort).apply(attrgetter('n'))

    # Realizar pivot da tabela
    cohort_pivot = df_cohort.pivot_table(index=['cohort'], columns='period_number', values='n_prof')

    # Obter o tamanho de cada grupo (cohort)
    cohort_size = cohort_pivot.iloc[:, 0]

    return cohort_pivot, cohort_size


def calcular_restantes(triangulo_cl, cohort_pivot):
    """
    O modelo chainladder calcula a quantidade de profissionais que saíram por mês.
    Essa função calcula, a partir do número inicial, quantos restaram, por meio de subtração
    :param triangulo_cl:
    :param cohort_pivot:
    :return:
    """
    dados = triangulo_cl.to_frame()

    if 192 not in dados:
        dados[192] = np.nan
    dados.drop([192], axis=1, inplace=True)
    dados.index = dados.index.strftime('%Y-%m')
    dados[0] = cohort_pivot[0].values[:-1]
    for col in dados.columns:
        if col != 0:
            dados[col] = (round(dados[0] - dados[col]))  # as typeint
        else:
            dados[col] = dados[col]  # astypeint
    dados = dados[[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180]]
    return dados


def calcular_percentual_restantes(restantes, cohort_size):
    return restantes.divide(cohort_size.values[:-1], axis=0)


def calcular_retencao_ano_regiao(percentual_restantes, regiao_saude):
    aux = percentual_restantes.melt(var_name='periodo', value_name='retention', ignore_index=False).reset_index().rename(
        columns={'index': 'cohort'})
    aux['cohort'] = pd.to_datetime(aux['cohort'])
    aux['a'] = aux.apply(lambda row: row['cohort'] + pd.DateOffset(months=row['periodo']), axis=1)
    aux2 = aux.loc[(aux['a'] <= '2024-01-01') & (aux['periodo'] != 0)].groupby('a').head(5)  # Considera os últimos 5 (diagonal)
    aux2 = aux2.loc[aux['a'] >= '2014-01-01']
    aux2['a'] = aux2['a'].dt.strftime('%Y-%m')
    aux2 = aux2.groupby('a')[['retention']].mean().reset_index().rename(columns={'a': 'ano'})
    aux2['regiao_saude'] = regiao_saude
    return aux2


def pre_processar_dados_retencao(percentual_restantes):
    df = percentual_restantes.reset_index().melt(id_vars='index')
    df['periodo'] = pd.to_datetime(df['index'], format='%Y-%m').dt.to_period('M') + df['variable'].astype(int).apply(
        pd.offsets.MonthEnd)
    df = df.set_index(['variable', 'index']).sort_values(['index', 'variable'])

    return df

## Funções auxiliares para modelagem dos dados

In [4]:
def instanciar_chainladder(cohort_pivot):
    triangulo = gerar_triangulo(cohort_pivot)

    cl_triangle = cl.Triangle(
        triangulo,
        origin="origin",
        development="development",
        columns="values",
        cumulative=True
    )

    return cl_triangle

## Funções auxiliares para visualização dos dados

In [5]:
def generate_retention_curve(dataframe, ax):
    tmp_df = dataframe.unstack()
    tmp_df.columns = tmp_df.columns.droplevel()
    tmp_df.plot(cmap='tab20', ax=ax)

    ax.legend(ncol=5)
    ax.set_ylim(0, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_title('Retenção percentual por período e cohort')
    ax.set_xlabel('períodos (meses)')


def generate_retention_matrix(dataframe, ax, percentual):
    if percentual:
        fmt = '.0%'
        titulo = 'percentual'
    else:
        fmt = 'g'
        titulo = 'absoluta'

    sns.heatmap(dataframe,
                mask=dataframe.isnull(),
                annot=True,
                fmt=fmt,
                annot_kws={"size": 8},
                cmap='RdYlGn',
                ax=ax)

    ax.set(xlabel='períodos (meses)', ylabel='')
    ax.set_title(f'Retenção {titulo} por período e cohort')


def gerar_graficos(restantes, percentual_restantes, titulo, nome_arquivo):
    # Plotar o gráfico
    fig = plt.figure(figsize=(15, 10))
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])

    generate_retention_matrix(dataframe=restantes.copy(), ax=ax1, percentual=False)

    # Matriz de Retenção com números percentuais
    generate_retention_matrix(dataframe=percentual_restantes.copy(), ax=ax2, percentual=True)

    # Curva de Retenção
    dados_retencao = pre_processar_dados_retencao(percentual_restantes.copy())
    generate_retention_curve(dataframe=dados_retencao.copy(), ax=ax3)

    plt.suptitle(titulo, fontsize=16)
    fig.tight_layout()
    plt.savefig(f'./imgs/{nome_arquivo}.png')
    plt.close()

## Obter dados

Os dados utilizados são provenientes do Cadastro Nacional de Estabelecimentos de Saúde - Profissionais (CNES-PF). Os arquivos disponibilizados pelo DATASUS na página https://datasus.saude.gov.br/transferencia-de-arquivos/ foram baixados e processados conforme procedimentos descritos a seguir:

- Coletar os dados sobre os profissionais
- Coletar competencia, código do município e CPF do profissional do CNES-PF de forma distinta.
- Ou seja, busca-se encontrar para cada ano e cada profissional, uma linha para cada município onde ele atuou. Entende-se o 'ano' como a competência 01 (janeiro).
- Nesse primeiro momento a análise está limitada apenas aos profissionais médicos e enfermeiros

O procedimento descrito foi realizado por meio da instrução SQL abaixo em infraestrutura própria:

```sql
SELECT DISTINCT
    COMPETEN,
    CODUFMUN,
    CASE
        WHEN pf.CBO LIKE '225%' OR pf.CBO LIKE '2231%' THEN 'Médico'
        WHEN pf.CBO LIKE '2235%' THEN 'Enfermeiro'
        END AS categoria,
    CPF_PROF
FROM Dados.cnes.PF
WHERE
  SUBSTR(COMPETEN, 1, 4) >= 2006
  AND SUBSTR(COMPETEN, 5, 2) = '01'
  AND (
        (pf.CBO LIKE '225%' OR pf.CBO LIKE '2231%') OR -- Medico
        (pf.CBO LIKE '2235%') -- Enfermeiro
    )
```

O exemplo traz consultas para médicos e enfermeiros. No entanto, para nosso artigo estamos focando apenas em médicos.

In [6]:
# df = pd.read_parquet('https://github.com/cigets-plataforma-observatorio/artigo_retencao/raw/main/dados/profissionais.parquet')
df = pd.read_parquet('dados/profissionais.parquet')
df.head()

,COMPETEN,CODUFMUN,categoria,CPF_PROF
0,201301,270010,Enfermeiro,{{<80>|{<83>|<81><80>}<80>
1,201301,270030,Enfermeiro,{~}<80><81><84>}<81>~
2,201301,270030,Médico,|}{{<82><81><82><84>
3,201301,270030,Enfermeiro,{|<80><84><83><84><84><84><83>
4,201301,270030,Médico,{{}<82>{|<80><84>|


Os dados sobre os municípios também foram coletados do DATASUS (https://datasus.saude.gov.br/transferencia-de-arquivos/), por meio do arquivo Base Territorial e tratado em infraestrutura própria. A consulta SQL mostra a junção das tabelas:

```sql
SELECT DISTINCT
    uf.DS_NOME as uf,
    uf.DS_sigla as uf_sigla,
    rs.CO_REGSAUD as cod_regsaud,
    rs.DS_NOME as regiao_saude,
    mun.CO_MUNICIP as cod_municipio,
    mun.CO_MUNICDV as cod_municipiodv,
    mun.DS_NOME as municipio
FROM "Dados.territorial"."tb_municip.parquet" mun
INNER JOIN "Dados.territorial"."tb_uf.parquet" uf on uf.CO_UF = mun.CO_UF
INNER JOIN "Dados.territorial"."rl_municip_regsaud.parquet" rsm on rsm.CO_MUNICIP = mun.CO_MUNICIP
INNER JOIN "Dados.territorial"."tb_regsaud.parquet" rs on rs.CO_REGSAUD = rsm.CO_REGSAUD
INNER JOIN "Dados.territorial"."rl_municip_macsaud.parquet" mrsm on mrsm.CO_MUNICIP = mun.CO_MUNICIP
INNER JOIN "Dados.territorial"."tb_regiao.parquet" mrs on mrs.CO_REGIAO = uf.CO_REGIAO
INNER JOIN "Dados.territorial"."rl_municip_macsaud.parquet" rlmac on rlmac.CO_MUNICIP = mun.CO_MUNICIP
INNER JOIN "Dados.territorial"."tb_macsaud.parquet" mac on CAST(rlmac.CO_MACSAUD AS VARCHAR) = mac.CO_MACSAUD
INNER JOIN "Dados.territorial"."lat_long.parquet" ll on CAST(mun.CO_MUNICIP AS VARCHAR) = ll.municipio
WHERE
    mun.CO_STATUS = 'ATIVO' AND
    uf.CO_STATUS = 'ATIVO' AND
    rs.CO_STATUS = 'ATIVO' AND
    mrs.CO_STATUS = 'ATIVO' AND
    mac.CO_STATUS = 'ATIVO'
```

In [7]:
# geo = pd.read_parquet('https://github.com/cigets-plataforma-observatorio/artigo_retencao/raw/main/dados/municipios.parquet')
geo = pd.read_parquet('dados/municipios.parquet')
geo['cod_regsaud'] = geo['cod_regsaud'].astype(str)
geo.head()

,uf,uf_sigla,cod_regsaud,regiao_saude,cod_municipiodv,cod_municipio,municipio
0,Rondônia,RO,11001,Vale do Jamari,1100452,110045,Buritis
1,Rondônia,RO,11003,Central,1100114,110011,Jaru
2,Rondônia,RO,11003,Central,1100254,110025,Presidente Médici
3,Rondônia,RO,11004,Madeira-Mamoré,1100205,110020,Porto Velho
4,Rondônia,RO,11004,Madeira-Mamoré,1100809,110080,Candeias do Jamari


In [8]:
# Realizar o join dos dados do CNES com tabela de municipios
df = df.merge(geo, left_on=['CODUFMUN'], right_on=['cod_municipio'])
df.head()

,COMPETEN,CODUFMUN,categoria,CPF_PROF,uf,uf_sigla,cod_regsaud,regiao_saude,cod_municipiodv,cod_municipio,municipio
0,201301,270010,Enfermeiro,{{<80>|{<83>|<81><80>}<80>,Alagoas,AL,27010,10ª Região de Saúde,2700102,270010,Água Branca
1,201601,270010,Enfermeiro,}{<83>|~<82>~}{,Alagoas,AL,27010,10ª Região de Saúde,2700102,270010,Água Branca
2,201301,270010,Enfermeiro,{|<84><81><84>~{|<80><81>,Alagoas,AL,27010,10ª Região de Saúde,2700102,270010,Água Branca
3,201601,270010,Médico,<83><82>{<81><83><84>{<80>,Alagoas,AL,27010,10ª Região de Saúde,2700102,270010,Água Branca
4,201301,270010,Enfermeiro,{~<82>~<83><80><83>}{,Alagoas,AL,27010,10ª Região de Saúde,2700102,270010,Água Branca


## Processar os dados e criar os gráficos e arquivos CSV de retenção por região de saúde

In [9]:
# Realizar processamento para cada categoria profissional
for categoria in ['Médico']:
#for categoria in df['categoria'].unique():
    df_categoria = df.loc[df['categoria'] == categoria].copy()

    # Criar listas para salvar dados a serem exportados
    triangulos = []
    retencao_ano = []
    retencao_geral = []

    # Realizar o processamento separado para cada região de saúde
    i=0
    for regiao_saude in df['cod_regsaud'].unique():
        try:
            print(f'Processando {regiao_saude} - {i+1}/{len(df["cod_regsaud"].unique())}')
            df_categoria_regiao = df_categoria.loc[df_categoria['cod_regsaud'] == regiao_saude].copy()

            # Agregar os dados por cohorts
            cohort_pivot, cohort_sizes = gerar_triangulo_agregado(df_categoria_regiao.copy())

            for fill_na in list(range(0, 181, 12)):
                if fill_na not in cohort_pivot.columns:
                    cohort_pivot[fill_na] = np.nan

            # Obter o chainladder
            triangulo_cl = instanciar_chainladder(cohort_pivot.copy())

            # Calcular profissionais restantes em cada período do cohort
            restantes = calcular_restantes(triangulo_cl, cohort_pivot.copy())

            # Realizar a divisão de cada célula da tabela pivotada pelo tamanho do grupo para deixar em percentual
            percentual_restantes = calcular_percentual_restantes(restantes, cohort_sizes)

            # Calcular retenção anual (últimos 5 anos) por região de saúde
            retencao_anual = calcular_retencao_ano_regiao(percentual_restantes.copy(), regiao_saude)
            retencao_ano.append(retencao_anual)

            # Calcular retenção geral (média de todo o triangulo)
            retencao_geral.append(pd.DataFrame({'regiao_saude': [regiao_saude], 'retencao_geral': [cohort_pivot.divide(cohort_sizes, axis = 0).unstack().mean()]}))

            # Salvar dados do triangulo para exportar em csv
            cohort_pivot['cod_regsaud'] = regiao_saude
            triangulos.append(cohort_pivot.copy())
            cohort_pivot.drop(['cod_regsaud'], axis=1, inplace=True)

            # Gerar gráficos
            nome_regiao = df_categoria_regiao["regiao_saude"].values[0]
            uf_regiao = df_categoria_regiao["uf_sigla"].values[0]
            gerar_graficos(restantes.copy(), percentual_restantes.copy(), f'Região de Saúde {nome_regiao} - {uf_regiao}', f'{categoria}{regiao_saude}')

            i+=1
        except Exception as e:
            print(f'Erro na região de saúde {regiao_saude}: {e}')
            print(traceback.format_exc())
            continue


    # Exportar dados em CSV
    pd.concat(triangulos).to_csv(f'csvs/{categoria}_triangulos.csv', sep=';')
    pd.concat(retencao_ano).to_csv(f'csvs/{categoria}_retencao_ano.csv', sep=';')
    pd.concat(retencao_geral).to_csv(f'csvs/{categoria}_retencao_geral.csv', sep=';')

Processando 27010 - 1/450
Processando 27007 - 2/450
Processando 27004 - 3/450
Processando 27003 - 4/450
Processando 27009 - 5/450
Processando 27001 - 6/450
Processando 27008 - 7/450
Processando 27006 - 8/450
Processando 27002 - 9/450
Processando 27005 - 10/450
Processando 13001 - 11/450
Processando 13006 - 12/450
Processando 13002 - 13/450
Processando 13004 - 14/450
Processando 13005 - 15/450
Processando 13009 - 16/450
Processando 13003 - 17/450
Processando 13007 - 18/450
Processando 13008 - 19/450
Processando 16001 - 20/450
Processando 16003 - 21/450
Processando 16002 - 22/450
Processando 32005 - 23/450
Processando 32004 - 24/450
Processando 32002 - 25/450
Processando 21019 - 26/450
Processando 21009 - 27/450
Processando 21002 - 28/450
Processando 21003 - 29/450
Processando 21004 - 30/450
Processando 21013 - 31/450
Processando 21014 - 32/450
Processando 21006 - 33/450
Processando 21008 - 34/450
Processando 21005 - 35/450
Processando 21001 - 36/450
Processando 21007 - 37/450
Processand